In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6efc03e3756fe1b607399aca65889bc2f6a9df78192b1fee54af6762996080a5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor


In [ ]:
spark = SparkSession.builder.appName("Colab").getOrCreate()

In [ ]:
df = spark.read.csv("/content/car_prices.csv",header=True,inferSchema=True)

In [ ]:
check_missig = df.select([sum(col(i).isNull().cast("int")).alias(i) for i in df.columns])
check_missig.show()

+----+----+-----+----+-----+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|year|make|model|trim| body|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|
+----+----+-----+----+-----+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|   0|7588| 7659|7917|10461|  0|    0|    11797|      88|  594|     594|     0|  7|           3|       3|
+----+----+-----+----+-----+---+-----+---------+--------+-----+--------+------+---+------------+--------+



In [ ]:
df = df.drop(df.columns[5])

In [ ]:
df = df.na.drop()

In [ ]:
df.show()

+----+---------+-------------------+--------------------+-----------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|year|     make|              model|                trim|       body|              vin|state|condition|odometer| color|interior|              seller|  mmr|sellingprice|            saledate|
+----+---------+-------------------+--------------------+-----------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|2015|      Kia|            Sorento|                  LX|        SUV|5xyktca69fg566472|   ca|        5|   16639| white|   black|kia motors americ...|20500|       21500|Tue Dec 16 2014 1...|
|2015|      Kia|            Sorento|                  LX|        SUV|5xyktca69fg561319|   ca|        5|    9393| white|   beige|kia motors americ...|20800|       21500|Tue Dec 16 2014 1...|
|2014|      BMW|           3 Series|          328i

In [ ]:
selected_features = ['year', 'odometer', 'make', 'model', 'trim', 'body', 'condition', 'color', 'interior']


In [ ]:
hasher = FeatureHasher(inputCols=selected_features, outputCol="features")
df_transformed = hasher.transform(df)

In [ ]:
from pyspark.ml.stat import Correlation

In [ ]:
# Calculate correlation matrix
correlation_matrix = Correlation.corr(df_transformed, "features").head()[0]

# Convert to Pandas DataFrame for better readability
import pandas as pd
corr_df = pd.DataFrame(correlation_matrix.toArray(), columns=selected_features, index=selected_features)
print(corr_df)

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
df_transformed.show()

ConnectionRefusedError: [Errno 111] Connection refused

----- Model  --

In [ ]:
train_data = df_transformed.sample(0.8,seed=41)
test_data = df_transformed.subtract(train_data)

In [ ]:
ln_reg = LinearRegression(featuresCol="features", labelCol="sellingprice")

In [ ]:
lm = ln_reg.fit(train_data)

In [ ]:
prediction = lm.transform(test_data)

In [ ]:
evaluator = RegressionEvaluator(labelCol="sellingprice", predictionCol="prediction", metricName="rmse")

In [ ]:
r_squared = evaluator.evaluate(prediction, {evaluator.metricName: "r2"})
print("R-squared:", r_squared)

R-squared: 0.7916480652546654


In [ ]:
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data:", rmse)

Root Mean Squared Error (RMSE) on test data: 5506.302844581191


In [ ]:
df.describe("sellingprice").show()

+-------+------------------+
|summary|      sellingprice|
+-------+------------------+
|  count|              5706|
|   mean|15531.962320364528|
| stddev|11945.861723709193|
|    min|               150|
|    max|            154000|
+-------+------------------+



----- Lasso Regression ----------

In [ ]:
#### Use Lasso Regression ###
lasso = LinearRegression(featuresCol="features", labelCol="sellingprice", regParam=0.1, elasticNetParam=1.0)
lass_reg= lasso.fit(train_data)

In [ ]:
model_lasso = lass_reg.transform(test_data)

In [ ]:
rmse_lasso = evaluator.evaluate(model_lasso)
print("Root Mean Squared Error (RMSE) on test data:", rmse_lasso)

Root Mean Squared Error (RMSE) on test data: 5503.384911502671


In [ ]:
lasso_r2 = evaluator.evaluate(model_lasso, {evaluator.metricName: "r2"})
print("R-squared:", lasso_r2)

R-squared: 0.791868828962514


--- Ridge Regression

In [ ]:
Ridge = LinearRegression(featuresCol="features", labelCol="sellingprice", regParam=0.1, elasticNetParam=1.0)
ridge_reg= Ridge.fit(train_data)

In [ ]:
ridge_model = ridge_reg.transform(test_data)

In [ ]:
rmse_ridge = evaluator.evaluate(ridge_model)
print("Root Mean Squared Error (RMSE) on test data:", rmse_ridge)

Root Mean Squared Error (RMSE) on test data: 5503.384911502671


In [ ]:
ridge_r2 = evaluator.evaluate(ridge_model, {evaluator.metricName: "r2"})
print("R-squared:", ridge_r2)

R-squared: 0.791868828962514
